In [ ]:
import torch
from torch_geometric.nn import Node2Vec

from ogb.nodeproppred import PygNodePropPredDataset

In [ ]:
dataset = PygNodePropPredDataset(name = "ogbn-products", root = 'dataset/')
data = dataset[0]
print(data)

In [ ]:
data.x

In [ ]:
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [ ]:
data.edge_index

In [ ]:
import networkx as nx

In [ ]:
from torch_geometric.utils import to_networkx

g = to_networkx(data, to_undirected=True)
nx.draw(g)

In [ ]:
x = data.x[train_idx]
edge_index = data.edge_index[train_idx]
small_data = torch_geometric.data.Data(x=x, edge_index=edge_index)

In [ ]:
data.num_node_features

# MLP

https://github.com/snap-stanford/ogb/blob/master/examples/nodeproppred/products/mlp.py

In [1]:
import torch
import torch.nn.functional as F
# from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [2]:
class MLP(torch.nn.Module):
    
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers, dropout):
        
        super(MLP, self).__init__()
        
        self.linears = torch.nn.ModuleList()        
        self.linears.append(torch.nn.Linear(in_dim, hidden_dim))
        for _ in range(1, num_layers-1):
            self.linears.append(torch.nn.Linear(hidden_dim, hidden_dim))
        self.linears.append(torch.nn.Linear(hidden_dim, out_dim))
        
        self.dropout = dropout
        
    
    def reset_parameters(self):
        
        for lin in self.linears:
            lin.reset_parameters()
            
    def forward(self, x):
        
        for lin in self.linears[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            
        x = self.linears[-1](x)
        x = torch.log_softmax(x, dim=-1)
        
        return x

In [3]:
def train(model, x, y_true, train_idx, optimizer, loss_fn):
    
    model.train()
    loss = 0
    
    optimizer.zero_grad()
    pred = model(x[train_idx])
    loss = loss_fn(pred, y_true.squeeze(1)[train_idx])
    
    loss.backward()
    optimizer.step()
    
    return loss.item()


@torch.no_grad()
def test(model, x, y_true, split_idx, evaluator):
    
    model.eval()
    
    out = model(x)
    y_pred = out.argmax(dim=-1, keepdim=True)
    
    train_acc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']]
    })['acc']
    
    valid_acc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']]
    })['acc']
    
    test_acc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']]
    })['acc']
    
    return train_acc, valid_acc, test_acc

In [4]:
def load_data(args):
    
    dataset = PygNodePropPredDataset(name='ogbn-products', root = 'dataset/')
    split_idx = dataset.get_idx_split()
    data = dataset[0]
    
    device = args['device']
    use_node_embedding = args['use_node_embedding']

    x = data.x
    if use_node_embedding:
        embedding = torch.load('embedding.pt', map_location='cpu')
        x = torch.cat([x, embedding], dim=-1)

    x = x.to(device)
    y_true = data.y.to(device)
    train_idx = split_idx['train'].to(device)
    
    return x, y_true, train_idx, split_idx, dataset.num_classes

In [5]:
def build_model(x, num_classes, args):
    
    device = args['device']
    
    if args['model_name'] == 'MLP':
        model = MLP(x.size(-1), args['hidden_dim'], num_classes, args['num_layers'], args['dropout']).to(device)
    
    return model

In [6]:
def run_model(model, x, y_true, train_idx, split_idx, args):

    evaluator = Evaluator(name='ogbn-products')

    model.reset_parameters()

    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
    loss_fn = F.nll_loss  # cross entropy = softmax + nll_loss，已經在 forward 裡面使用 softmax，因此這邊只用 nll_loss

    best_model = None
    best_valid_acc = 0

    for epoch in range(args['epochs']):
        loss = train(model, x, y_true, train_idx, optimizer, loss_fn)
        result = test(model, x, y_true, split_idx, evaluator)

        if epoch % args['log_steps'] == 0:
            train_acc, valid_acc, test_acc = result
            print(f'Epoch: {epoch:02d}, '
                  f'Loss: {loss:.4f}, '
                  f'Train: {100 * train_acc:.2f}%, '
                  f'Valid: {100 * valid_acc:.2f}%, '
                  f'Test: {100 * test_acc:.2f}%')

In [7]:
def main():
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)

    args = {
        'device': device,
        'num_layers': 3,
        'hidden_dim': 256,
        'dropout': 0.0,
        'lr': 0.01,
        'epochs': 300,
        'log_steps': 10,
        'use_node_embedding': False,
        'model_name': 'MLP'
    }
    
    x, y_true, train_idx, split_idx, num_classes = load_data(args)
    
    model = build_model(x, num_classes, args)
    run_model(model, x, y_true, train_idx, split_idx, args)

In [8]:
main()

Epoch: 00, Loss: 3.8492, Train: 30.84%, Valid: 30.83%, Test: 26.94%
Epoch: 10, Loss: 1.7212, Train: 55.96%, Valid: 55.10%, Test: 43.27%
Epoch: 20, Loss: 1.3275, Train: 64.73%, Valid: 63.70%, Test: 51.65%
Epoch: 30, Loss: 1.1348, Train: 69.13%, Valid: 67.91%, Test: 55.11%
Epoch: 40, Loss: 1.0077, Train: 71.87%, Valid: 70.37%, Test: 57.07%
Epoch: 50, Loss: 0.9154, Train: 74.16%, Valid: 72.15%, Test: 58.40%
Epoch: 60, Loss: 0.8406, Train: 75.95%, Valid: 73.40%, Test: 59.58%
Epoch: 70, Loss: 0.7805, Train: 77.43%, Valid: 74.15%, Test: 60.11%
Epoch: 80, Loss: 0.7400, Train: 78.52%, Valid: 74.58%, Test: 60.46%
Epoch: 90, Loss: 0.6969, Train: 79.64%, Valid: 74.95%, Test: 60.76%
Epoch: 100, Loss: 0.6725, Train: 80.24%, Valid: 74.95%, Test: 60.85%
Epoch: 110, Loss: 0.6409, Train: 81.08%, Valid: 75.20%, Test: 61.01%
Epoch: 120, Loss: 0.6163, Train: 81.74%, Valid: 75.25%, Test: 60.92%
Epoch: 130, Loss: 0.5999, Train: 82.06%, Valid: 75.10%, Test: 60.98%
Epoch: 140, Loss: 0.5837, Train: 82.89%, Val